# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data_preprocess import series_to_supervised,load_data

import numpy as np

In [3]:
#r='data/instant_data/rain_small.csv'
#w='data/instant_data/water_small.csv'
r = "data/instant_data/rain.csv"
w = 'data/instant_data/water.csv'
rw = load_data(r,w)
df =rw.df.resample('d').mean()

MemoryError: Unable to allocate 4.94 GiB for an array with shape (929, 714384) and data type float64

In [3]:
TARGET = 'CPY015_w'

## Period 2013-2015 ##
X_in = df['2013-01-01':'2015-01-01'].interpolate(limit=15)
X_in = X_in.astype('float32')
X = X_in.drop([TARGET],axis=1)
xlabels = list(X.columns)


# REPLACE NAN WITH 0
X = X.fillna(0).values
y = np.log(X_in[TARGET].fillna(0).values)
where_are_NaNs = np.isnan(y)
y[where_are_NaNs] = 0


# MARS feature selection (from all stations)

In [4]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.296832  1      0.298  0.000  0.000  
1     0       935  364   0.144594  3      0.147  0.513  0.506  
2     0       1016  544   0.118268  5      0.122  0.602  0.590  
3     0       1015  516   0.110742  7      0.116  0.627  0.611  
4     0       873  591   0.104027  9      0.110  0.650  0.630  
5     0       715  554   0.097734  11     0.105  0.671  0.647  
6     0       882  501   0.093214  13     0.102  0.686  0.658  
7     0       934  -1    0.089119  14     0.098  0.700  0.671  
8     0       998  296   0.084469  16     0.094  0.715  0.684  
9     0       428  558   0.082088  18     0.093  0.723  0.688  
10    0       879  437   0.079751  20     0.091  0.731  0.693  
11    0       935  514   0.077447  22     0.090  0.739  0.697  
12    0       9

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [ ]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [6]:
print(model.summary_feature_importances(sort_by='gcv')[:2000])

             rss    gcv    nb_subsets
BKK006_w     0.53   0.63   0.01          
BKK007_w     0.12   0.13   0.05          
DNP024_w     0.03   0.03   0.02          
GLF002_w     0.03   0.02   0.02          
TB0003_r     0.02   0.02   0.02          
WEI061_r     0.02   0.02   0.02          
WEI131_r     0.01   0.01   0.02          
TB0002_w     0.01   0.01   0.02          
NKO002_w     0.01   0.01   0.02          
CPY009_w     0.01   0.01   0.02          
DIV003_r     0.01   0.01   0.02          
SACS_r       0.01   0.01   0.02          
PIN001_w     0.01   0.01   0.01          
BKK002_w     0.01   0.01   0.02          
WEI051_r     0.01   0.00   0.01          
DNP004_r     0.01   0.00   0.02          
CHI005_r     0.00   0.00   0.01          
PDAM_r       0.00   0.00   0.01          
PANC_r       0.00   0.00   0.01          
BLUG_r       0.00   0.00   0.02          
RCHM_r       0.01   0.00   0.02          
BSDE_r       0.00   0.00   0.02          
KWN001_r     0.01   0.00   0.02       

# MARS feature selection (Times interval perspective)

In [7]:
n_timelag=7

reframed_data = series_to_supervised(X_in.iloc[:20,:], n_in= 3,n_out=1)
# not work
X_in.shape,reframed_data.shape

((731, 1245), (0, 4980))

In [8]:
def to_sub(df,n_in=5,n_out=3):
    df_in = df.shift(n_in)
    df_out = df.shift(-n_out)
    return df_in

In [9]:
test = X_in.iloc[:,:5]

print(test.head())
print(to_sub(test).head())

            ACRU_r  ANLI_r  BAKI_r  BARI_r  BBHN_r
date                                              
2013-01-01     0.0     0.0     0.0     0.0     0.0
2013-01-02     0.0     0.0     0.0     0.0     0.0
2013-01-03     0.0     0.0     0.0     0.0     0.0
2013-01-04     0.0     0.0     0.0     0.0     0.2
2013-01-05     0.0     0.0     0.0     0.0     0.2
            ACRU_r  ANLI_r  BAKI_r  BARI_r  BBHN_r
date                                              
2013-01-01     NaN     NaN     NaN     NaN     NaN
2013-01-02     NaN     NaN     NaN     NaN     NaN
2013-01-03     NaN     NaN     NaN     NaN     NaN
2013-01-04     NaN     NaN     NaN     NaN     NaN
2013-01-05     NaN     NaN     NaN     NaN     NaN


In [11]:
lookback= 7

test_size=int(.3 * len(test))
X=[]
y=[]


In [12]:
for i in range(len(test)-lookback-1):
    t=[]
    for j in range(0,lookback):
        
        t.append(test[[(i+j)], :])
    X.append(t)
    y.append(test[i+ lookback,1])

TypeError: '([0], slice(None, None, None))' is an invalid key